In [1]:
import cv2
import numpy as np
import math
from math import pi,atan
import os
from tqdm import tqdm

# 源文件夹路径
source_folder = r"C:\Users\wac\Desktop\fx\img\fisheye_sky"
# 目标文件夹路径
save_folder = r"C:\Users\wac\Desktop\fx\img\fisheye_sky2"

if not os.path.exists(save_folder):
    os.mkdir(save_folder)
    
for file_name in tqdm(os.listdir(source_folder)):
    if file_name.endswith(".jpg"):
        # 1.全景图像转化为鱼眼图像
        imgPath = os.path.join(source_folder,file_name)
        img = cv2.imread(imgPath)
        img = img[0:400, :]
        height,width = img.shape[:2]
        cx = width/(2*math.pi)
        cy = width/(2*math.pi)
        img_new = np.zeros((int(cx+1)*2,int(cx+1)*2,3),dtype=np.uint8)

        for y in range(img_new.shape[1]):
            for x in range(img_new.shape[0]):
                if x < cx:
                    theta = pi / 2 + atan((y - cy)/(x - cx))
                else:
                    theta = (pi*3) / 2 + atan((y - cy)/(x - cx))
                r = np.sqrt((x - cx)**2 + (y - cy)**2)
                x1 = (theta*width) / (2*pi)
                y1 = (r*height) / cx
                img_new[x][y] = cv2.getRectSubPix(img,(1,1),(x1,y1))
                
        mask = np.zeros_like(img_new)
        mask = cv2.circle(mask,(int(cx+1),int(cx+1)),int(cx+1),(255,255,255),-1)
        result = cv2.bitwise_and(img_new,mask)
        
        #保存进行方向校正之前的鱼眼图像
        file_name_without_extension = os.path.splitext(file_name)[0]
        savePath = os.path.join(save_folder, f"{file_name_without_extension}_fisheye.jpg")
        cv2.imwrite(savePath, result)

        # 2.鱼眼图方向矫正
        # 对旋转后的图像（dst）进行水平镜像翻转
        img_flipped = cv2.flip(result, 1)  # 参数 1 表示水平翻转

        # 计算旋转中心点
        height, width = img_flipped.shape[:2]
        center_x, center_y = width // 2, height // 2

        # 设置旋转角度
        file_name = file_name.replace(".jpg", "")
        split_names = file_name.split("_")
        northRotation = int(split_names[5]) # 正北方向得角度
        #northRotation = 90
        angle = (540 - northRotation)%360 - northRotation  # 旋转角度 = 摄影师的方向[(360+180-正北的方向)%360]-正北的方向

        # 计算仿射变换矩阵
        M = cv2.getRotationMatrix2D((center_x, center_y), angle, 1)

        # 进行仿射变换
        dst = cv2.warpAffine(img_flipped, M, (width, height))

        file_name = file_name+".jpg"
        savePath = os.path.join(save_folder,file_name)
        cv2.imwrite(savePath,dst)

100%|██████████| 1/1 [00:02<00:00,  2.42s/it]
